# Modelo ViT

Implementação do Visual Transformer. Referências:

* [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
* [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)
* https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py

### Configurações iniciais

In [1]:
import torch
from torch import nn

class PatchifyLayer(nn.Module):
    """Módulo que transforma uma imagem em um conjunto de tokens. Mesmo
    módulo implementado no notebook anterior."""
        
    def __init__(self, image_size, patch_size, token_dim):
        super().__init__()
        self.conv_proj = nn.Conv2d(
            3, token_dim, kernel_size=patch_size, stride=patch_size
        )
        new_size = image_size//patch_size
        seq_length = new_size**2
        self.token_dim = token_dim
        self.new_size = new_size
        self.seq_length = seq_length

    def forward(self, x):
        x = self.conv_proj(x)
        x = x.reshape(x.shape[0], self.token_dim, -1)
        x = x.permute(0, 2, 1)

        return x
    
# Parâmetros de teste. Os nomes utilizados são os mesmos do código-fonte do Pytorch
bs = 8             # batch size
image_size = 224   # tamanho da imagem
patch_size = 16    # tamanho dos patches 16x16 para gerar tokens
num_layers = 12    # número de camadas
num_heads = 12     # número de cabeças para a multihead attention
token_dim = 768    # dimensão de cada token
mlp_dim = 3072     # dimensão da camada linear após a atenção
seq_length = (image_size//patch_size)**2  # tamanho de cada sequência

### Camada linear

A camada linear implementada abaixo será utilizada após cada camada de atenção. Ela é uma camada simples formada por linear->relu->linear. As camadas lineares incluem uma expansão de canais, ou seja, o número de canais é aumentado na primeira camada e reduzido na segunda.

In [2]:
class MLP(nn.Module):
    """Camada multilayer perceptron / feedforward. 
    Nota: Usualmente mlp_dim>token_dim."""

    def __init__(self, token_dim, mlp_dim):
        super().__init__()

        self.layers = nn.Sequential(
            torch.nn.Linear(token_dim, mlp_dim),
            nn.ReLU(),
            torch.nn.Linear(mlp_dim, token_dim),
        )

    def forward(self, x):
        return self.layers(x)

x = torch.rand(bs, seq_length, token_dim)
mlp = MLP(token_dim, mlp_dim)
out = mlp(x)
out.shape

torch.Size([8, 196, 768])

### Bloco do codificador

Um transformer consiste em uma sequência de blocos de codificação. Esses blocos são equivalentes ao ResidualBlock que implementamos para a ResNet (conv->batchnorm->relu->conv->batchnorm->relu), mas no caso do transformer temos layernorm->attention->layernorm->mlp

A camada LayerNorm faz o mesmo papel do BatchNorm. Poderíamos ter usado BatchNorm, mas na prática LayerNorm tende a funcionar melhor com transformers.

In [3]:
class EncoderBlock(nn.Module):
    """Bloco codificador de um transformer."""

    def __init__(self, num_heads, token_dim, mlp_dim):
        super().__init__()

        # Normalização e atenção
        self.ln_1 = nn.LayerNorm(token_dim)
        self.attention = nn.MultiheadAttention(token_dim, num_heads, batch_first=True)

        # Normalização e camada linear
        self.ln_2 = nn.LayerNorm(token_dim)
        self.mlp = MLP(token_dim, mlp_dim)

    def forward(self, input):

        x = self.ln_1(input)
        x, _ = self.attention(x, x, x)
        # Adiciona resíduo (assim como na resnet)
        x = x + input   
        
        y = self.ln_2(x)
        y = self.mlp(y)

        # Adciona resíduo e retorna
        return x + y
    
x = torch.rand(bs, seq_length, token_dim)
eb = EncoderBlock(num_heads, token_dim, mlp_dim)
out = eb(x)
out.shape
    

torch.Size([8, 196, 768])

### Alguns conceitos extra

#### Token de classe

O codificador do transformer realiza a mistura entre os tokens da sequência de entrada. Mas para aplicarmos o modelo em tarefas de classificação, precisamos de alguma forma extrair atributos da sequência como um todo, pois queremos caracterizar a imagem, e não os tokens. Por exemplo, para uma sequência de tamanho 1 x 196 x 768 representando uma imagem, queremos extrair um tensor de tamanho 1 x 768 contendo 768 atributos para toda a imagem. 

Uma forma de fazer isso seria simplesmente calcular a média dos valores:

In [4]:
# Imagem de exemplo
x = torch.rand(bs, seq_length, token_dim)
features = x.mean(dim=1)
# 768 atributos para cada imagem
features.shape

torch.Size([8, 768])

Mas ao invés de obter a média é comum ser feito um outro procedimento. Um token especial responsável por caracterizar a imagem como um todo é adicionado à sequência. Esse token é um parâmetro *treinável* do modelo, ou seja, ele é modificado durante o treinamento para reduzir a loss function. **Apenas o valor desse token é usado em tarefas de classificação, o restante da sequência é descartado na saída do transformer**. Esse token especial será "misturado" com os outros tokens ao longo do transformer (por causa da atenção) e também otimizado para resultar na melhor classificação possível.

In [5]:
# Token especial de classe
class_token = nn.Parameter(torch.zeros(1, 1, token_dim))

Dentro do método .forward() do transformer teremos:

In [6]:
# bs imagens representadas por seq_length tokens de dimensão token_dim
x = torch.rand(bs, seq_length, token_dim)

# Expansão do token de classe de 1 x 1 x token_dim -> bs x 1 x token_dim
# Os valores são copiados para serem iguais ao longo do batch
batch_class_token = class_token.expand(bs, -1, -1)
# Concatena na dimensão da sequência
x = torch.cat([batch_class_token, x], dim=1)

#### Positional embedding

Uma camada de atenção é equivariante à permutação da sequência de entrada. Isso quer dizer que se reordenarmos a sequência de entrada, o resultado da camada também será reordenado da mesma forma:

In [10]:
x = torch.rand(bs, seq_length, token_dim)
# Reordena os tokens da sequência de forma aleatória
inds = torch.randperm(seq_length)
x_p = x[:, inds]
# Aplica a camada de atenção nas duas versões da sequência
mha = nn.MultiheadAttention(num_heads=num_heads, embed_dim=token_dim)
out = mha(x, x, x)[0]
out_p = mha(x_p, x_p, x_p)[0]
# Compara o resultado original permutado com o resultado usando a entrada permutada
torch.allclose(out[:, inds], out_p)

True

Com isso, a rede não terá qualquer informação sobre a posição original dos tokens na imagem. Por exemplo, o resultado para um patch no canto esquerdo inferior da imagem será o mesmo se o patch for inserido no centro da imagem. Em geral, é importante a rede ter informação sobre a posição do patch na imagem, de forma que essa informação possa ser usada no mecanismo de atenção. Para adiciona tal informação, usamos o chamado positional embedding, que é um tensor que possuirá informação sobre a posição de cada token na imagem.

Um possível tensor com essa informação pode ser:

In [7]:
pos_embedding = torch.linspace(0, 1, seq_length*token_dim).reshape(seq_length, token_dim)

Nesse caso, para cada token `pos_embedding` possui um valor fixo que caracteriza unicamente cada token. Por exemplo, o token 10 terá o código pos_embedding[10] associado a ele. Tendo esse código, a prática comum é simplesmente somar o código aos tokens sendo processados:

In [8]:
# Sequência
x = torch.rand(bs, seq_length, token_dim)
# Adição da posição dos tokens
x = x + pos_embedding

A ideia é que a rede poderá utilizar esse valor que foi somado como referência sobre a posição do token. Transformers podem ser treinados sem positional encoding, e em geral dão bons resultados. Mas a adição de posição tende a levar a uma pequena melhora na performance. 

Existem várias outras possíveis codificações para posição dos tokens. Curiosamente, atualmente é prática comum simplesmente utilizar um parâmetro treinável como positional encoding:

In [9]:
# Usamos uma distribuição normal ao invés de torch.zeros para que os
# valores inciais não sejam todos iguais. Essa estratégia foi usada no
# artigo do modelo BERT
pos_embedding = torch.normal(mean=0., std=0.02, size=(1, seq_length, token_dim))
pos_embedding = nn.Parameter(pos_embedding)

### Transformer

Finalmente podemos o modelo transformer

In [10]:
class VisionTransformer(nn.Module):

    def __init__(self, image_size, patch_size, num_layers, num_heads, token_dim,
        mlp_dim, num_classes):
        super().__init__()

        # Transforma imagem em tokens
        self.patchify = PatchifyLayer(image_size, patch_size, token_dim)
        # Tamanho da sequência
        seq_length = (image_size//patch_size)**2

        # Adiciona token para a classe
        self.class_token = nn.Parameter(torch.zeros(1, 1, token_dim))
        seq_length += 1

        # Informação sobre a posição
        self.pos_embedding = nn.Parameter(torch.empty(1, seq_length, token_dim).normal_(std=0.02))  

        # Codificador
        encoder = []
        for _ in range(num_layers):
            encoder.append(EncoderBlock(num_heads, token_dim, mlp_dim))
        self.encoder = nn.Sequential(*encoder)
        self.ln = nn.LayerNorm(token_dim)

        # Camada de classificação
        self.final = nn.Linear(token_dim, num_classes)

    def forward(self, x):

        # bs x c x H x W -> bs x seq_length x token_dim
        x = self.patchify(x)
        bs = x.shape[0]

        # Expansão do token de classe de 1 x 1 x token_dim -> bs x 1 x token_dim
        batch_class_token = self.class_token.expand(bs, -1, -1)
        # Concatena na dimensão da sequência
        x = torch.cat([batch_class_token, x], dim=1)

        # Adiciona embedding posicional
        x = x + self.pos_embedding
        x = self.ln(self.encoder(x))

        # Extrai apenas o token de classe de cada batch
        x = x[:, 0]

        # Usa o token de classe para classificar a imagem
        # bs x token_dim -> bs x num_classes
        x = self.final(x)

        return x
    
vit = VisionTransformer(image_size, patch_size, num_layers, num_heads, 
                        token_dim, mlp_dim, num_classes=1000)

x = torch.rand(bs, 3, image_size, image_size)
out = vit(x)
out.shape

torch.Size([8, 1000])

### Modelo do Pytorch

In [11]:
from torchvision.models import vision_transformer

model = vision_transformer.vit_b_16()

print(f'{model.image_size=}')
print(f'{model.patch_size=}')
print(f'num_layers={len(model.encoder.layers)}')
print(f'num_heads={model.encoder.layers[0].num_heads}')
print(f'{model.hidden_dim=}')
print(f'{model.mlp_dim=}')
print(f'{model.seq_length=}')
print(f'{model.num_classes=}')

model.image_size=224
model.patch_size=16
num_layers=12
num_heads=12
model.hidden_dim=768
model.mlp_dim=3072
model.seq_length=197
model.num_classes=1000


Se desejarmos utilizar o transformer para outra tarefa além de classificação, podemos extrair os atributos gerados pelo modelo do Pytorch assim como fizemos para ResNets. Basta fazermos:

In [12]:
model.heads = nn.Identity()
out = model(x)
# 768 atributos extraídos de cada imagem do batch
out.shape

torch.Size([8, 768])